<a href="https://colab.research.google.com/github/gramelon2003/Python_ptdl/blob/master/Ph%C3%A2n_t%C3%ADch_th%E1%BB%91ng_k%C3%AA_suy_di%E1%BB%85n_ANOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chuẩn bị dữ liệu

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
uploaded = files.upload()

Saving Diet_R.csv to Diet_R.csv


In [ ]:
diet = pd.read_csv("Diet_R.csv")
diet

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks
0,25,,41,171,60,2,60.0
1,26,,32,174,103,2,103.0
2,1,0,22,159,58,1,54.2
3,2,0,46,192,60,1,54.0
4,3,0,55,170,64,1,63.3
...,...,...,...,...,...,...,...
73,74,1,35,183,83,3,80.2
74,75,1,49,177,84,3,79.9
75,76,1,28,164,85,3,79.7
76,77,1,40,167,87,3,77.8


QUY TRÌNH PHÂN TÍCH ANOVA MỘT CHIỀU

Tiến hành kiểm tra các giả định:

- Các mẫu độc lập với nhau
- Các mẫu có cùng phương sai
- Các mẫu đều tuân theo phân phối chuẩn

Xây dựng giả thuyết

$$H_{0}: \mu_{0} = \mu_{1} = ... = \mu_{n}$$
$$H_{a}: \text{Có tối thiểu một cặp giá trị trung bình có giá trị khác nhau}$$

Giá trị thống kê F được sử dụng trong việc ra quyết định của phân tích ANOVA một chiều. Công thức tính như sau:

$$F = \frac{MS_{B}}{MS_{W}}$$

với:

- $MS_{B} = \frac{\sum_{k}n_{k}(\overline{\rm x_{k}} - \overline{\rm x})^{2}}{k - 1}$
- $MS_{W} = \frac{\sum_{k}n_{i,k}( x_{i,k} - \overline{\rm x})^{2}}{k - 1}$

Loại bỏ giả thuyết $H_{0}$ khi chỉ số F > chỉ số F-critical (hoặc giá trị p tương ứng < 0.05(có thể thay đổi))

Phân tích ANOVA chấp nhận các giả định sau:

(1) Tất cả các nhóm đều độc lập với nhau

(2) Dữ liệu của mỗi nhóm đều tuân theo phân phối chuẩn. Tính chất này có thể được kiểm định bằng phương pháp Shapiro-Wilks.

(3) Phương sai (variance) của các nhóm bằng nhau. Tính chất này có thể được kiểm định bằng phương pháp kiểm định Bartlett hay Levene.

Nếu dữ liệu hiện có thỏa mãn các tính chất trên thì chúng ta mới tiến hành kiểm định ANOVA. Do vậy trước khi tiến hành kiểm định ANOVA, chúng ta cần thực hiện kiểm tra tập dữ liệu cần thỏa mãn 3 tính chất trên.

Phát biểu câu hỏi nghiên cứu:

Các nhà khoa học đang nghiên cứu về tác động của những phương thức ăn kiên lên cân nặng của con người. Họ tiến hành nghiên cứu trên 78 tình nguyện viên với 3 phương thức ăn kiên. Các nhà khoa học cho rằng không có sự khác biệt về số kg cân nặng giảm giữa ba nhóm được nghiên cứu. Hãy tiến hành kiểm định ý kiến trên từ tập số liệu Diet_R.csv

Đặt giả thuyết:
$$H_{0}: \mu_{0} = \mu_{1} = \mu_{2}$$
$$H_{a}: \text{Có tối thiểu một cặp giá trị trung bình có giá trị khác nhau}$$

Ta sẽ tiến hành phân tích ANOVA 1 chiều với tập dữ liệu Diet_R.csv

Lọc bớt các cột không cần thiết

In [ ]:
diet_1 = diet.drop(["Person", "gender", "Age", "Height"], axis = 1)

In [ ]:
diet_1

,pre.weight,Diet,weight6weeks
0,60,2,60.0
1,103,2,103.0
2,58,1,54.2
3,60,1,54.0
4,64,1,63.3
...,...,...,...
73,83,3,80.2
74,84,3,79.9
75,85,3,79.7
76,87,3,77.8


In [ ]:
pre_weight = diet_1.iloc[:, 0]
diet_group = diet_1.iloc[:, 1]
weight6weeks = diet_1.iloc[:, 2]

In [ ]:
a = []
for i in range(0, len(pre_weight)):
    b = pre_weight[i] - weight6weeks[i]
    c = round(b, 1)
    a.append(c)
loseweight = pd.Series(a, name = "loseweight")

In [ ]:
diet_2 = pd.concat([loseweight, diet_group], axis = 1)
diet_2

,loseweight,Diet
0,0.0,2
1,0.0,2
2,3.8,1
3,6.0,1
4,0.7,1
...,...,...
73,2.8,3
74,4.1,3
75,5.3,3
76,9.2,3


Chuyển đổi giá trị từ định lượng sang định tính

In [ ]:
diet_2["Diet"].replace({1: 'group_1', 2: 'group_2', 3: 'group_3'}, inplace= True)
diet_2.head()

,loseweight,Diet
0,0.0,group_2
1,0.0,group_2
2,3.8,group_1
3,6.0,group_1
4,0.7,group_1


Bắt đầu chuyển đổi dữ liệu

In [ ]:
group_1 = group_2 = group_3 = []
group_1 = diet_2['loseweight'][diet_2['Diet'] == 'group_1']
group_2 = diet_2['loseweight'][diet_2['Diet'] == 'group_2']
group_3 = diet_2['loseweight'][diet_2['Diet'] == 'group_3']
print(group_1.head())

2    3.8
3    6.0
4    0.7
5    2.9
6    2.8
Name: loseweight, dtype: float64


In [ ]:
print(group_2.head())

0     0.0
1     0.0
16   -2.1
17    2.0
18    1.7
Name: loseweight, dtype: float64


In [ ]:
print(group_3.head())

30    7.0
31    5.6
32    3.4
33    6.8
34    7.8
Name: loseweight, dtype: float64


Tiến hành kiểm tra giả định dữ liệu của các nhóm đều tuân theo phân phối chuẩn

In [ ]:
sta_value1, p_value1 = stats.shapiro(group_1)
sta_value2, p_value2 = stats.shapiro(group_2)
sta_value3, p_value3 = stats.shapiro(group_3)
if p_value1 < 0.05:
  print("Có bằng chứng bác bỏ giả thuyết nhóm thứ nhất không tuân theo phân phối chuẩn")
else:
  print("Không có bằng chứng bác bỏ giả thuyết nhóm thứ nhất không tuân theo phân phối chuẩn. Từ đó nhóm 1 có thể tuân theo phân phối chuẩn")
if p_value2 < 0.05:
  print("Có bằng chứng bác bỏ giả thuyết nhóm thứ nhất không tuân theo phân phối chuẩn")
else:
  print("Không có bằng chứng bác bỏ giả thuyết nhóm thứ nhất không tuân theo phân phối chuẩn. Từ đó nhóm 2 có thể tuân theo phân phối chuẩn")
if p_value3 < 0.05:
  print("Có bằng chứng bác bỏ giả thuyết nhóm thứ nhất không tuân theo phân phối chuẩn")
else:
  print("Không có bằng chứng bác bỏ giả thuyết nhóm thứ nhất không tuân theo phân phối chuẩn. Từ đó nhóm 3 có thể tuân theo phân phối chuẩn")

Không có bằng chứng bác bỏ giả thuyết nhóm thứ nhất không tuân theo phân phối chuẩn. Từ đó nhóm 1 có thể tuân theo phân phối chuẩn
Không có bằng chứng bác bỏ giả thuyết nhóm thứ nhất không tuân theo phân phối chuẩn. Từ đó nhóm 2 có thể tuân theo phân phối chuẩn
Không có bằng chứng bác bỏ giả thuyết nhóm thứ nhất không tuân theo phân phối chuẩn. Từ đó nhóm 3 có thể tuân theo phân phối chuẩn


Các nhóm đều tuân theo phân phối chuẩn. Vì vậy chúng ta sẽ tiến hành kiểm tra giả định các nhóm có phương sai bằng nhau hay không.

Đặt giả thuyết:

$H_0$: "Không có sự khác biệt về phương sai của các nhóm"

$H_a$: "Có sự khác biệt về phương sai của các nhóm"

In [ ]:
stat_value, p_value = stats.levene(group_1, group_2, group_3)
if p_value < 0.05:
  print("có bằng chứng bác bỏ giả thuyết H0")
else:
  print("Không có bằng chứng bác bỏ giả thuyết H0")

Không có bằng chứng bác bỏ giả thuyết H0


Vì các nhóm dữ liệu thỏa mãn tính chất để tiến hành kiểm định ANOVA nên chúng ta sẽ tiến hành kiểm định ANOVA 1 chiều

Phát biểu giả thuyết:

$H_0$: Không có sự khác biệt về giá trị trung bình giữa các nhóm

$H_a$: Tồn tại có sự khác biệt về giá trị trung bình giữa các nhóm

In [ ]:
sta_value, p_value = stats.f_oneway(group_1, group_2, group_3)
if p_value < 0.05:
  print("có bằng chứng bác bỏ giả thuyết H0")
else:
  print("Không có bằng chứng bác bỏ giả thuyết H0")

có bằng chứng bác bỏ giả thuyết H0


Kết quả kiểm định là có sự khác biệt giữa các nhóm ăn kiên 1, 2 và 3. Từ đó kết luận là các nhóm ăn kiên theo các phương pháp khác nhau dẫn đến việc giảm cân khác nhau